# Pandas 피봇과 그룹 연산

## 피봇 테이블

피봇 테이블(pivot table)이란 데이터 열(column) 중에서 두 개를 키(key)로 사용하여 데이터를 선택하는 방법을 말한다.

피봇 테이블을 사용하기 위해서는 키가 될 수 있는 두 개의 열(column) 혹은 필드(field)를 선택하여 이 두 열을

- 행 인덱스 (row index)
- 열 인덱스 (column index)

로 변경해야 한다.

https://datascienceschool.net/upfiles/f281a1a51f834b72add6abc73f9c95a0.png

- pivot 메서드를 사용하면 행 인덱스, 열 인덱스, 자료가 될 3가지의 열(column)을 지정할 수 있다.

In [2]:
import numpy as np
import pandas as pd

In [3]:
data = {
    'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
    'year': [2000, 2001, 2002, 2001, 2002],
    'pop': [1.5, 2.5, 3.0, 2.5, 3.5]
}
df = pd.DataFrame(data, columns=["state", "year", "pop"])
df

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,2.5
2,Ohio,2002,3.0
3,Nevada,2001,2.5
4,Nevada,2002,3.5


In [4]:
df.pivot("state", "year", "pop")

year,2000,2001,2002
state,,,
Nevada,NaN,2.5,3.5
Ohio,1.5,2.5,3.0


- 행 인덱스와, 열 인덱스가 될 자료는 키(key)의 역할을 해야 한다. 즉, 이 값으로 데이터가 유일하게(unique) 결정되어야 한다.

In [6]:
df.pivot("year", "pop", "state")

ValueError: Index contains duplicate entries, cannot reshape

In [7]:
df.set_index(["state", "year"]).unstack()

pop          
year   2000 2001 2002
state                
Nevada  NaN  2.5  3.5
Ohio    1.5  2.5  3.0

## 그룹 연산

그룹 연산은 피봇 테이블과 달리 키에 의해서 결정되는 데이터가 복수개가 있어도 괜찮다. 대신 연산을 통해 복수개의 그룹 데이터에 대한 대표값을 정한다. 이를 split-apply-combine 연산이라고도 한다.

- split 단계
    - 특정 Key 값에 따라 데이터 그룹을 만든다.
- apply 단계
    - 각각의 그룹에 대해 원하는 연산을 하여 대표값을 생성한다.
        - count(), mean(), median(), min(), max()
        - sum(), prod(), std(), var(), quantile()
        - first(), last()
- combine 단계
    - 그룹의 Key 값에 대해 원하는 연산의 결과를 Value로 지정한 dict를 생성한다.
    
https://datascienceschool.net/upfiles/5cf33c481e8041ebbf56a5af1f84d487.png

In [8]:
np.random.seed(0)
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,data1,data2,key1,key2
0,1.764052,-0.977278,a,one
1,0.400157,0.950088,a,two
2,0.978738,-0.151357,b,one
3,2.240893,-0.103219,b,two
4,1.867558,0.410599,a,one


- 문제: key1 값에 따른 data1의 평균은?

In [9]:
df.data1.groupby(df.key1).mean()

key1
a    1.343923
b    1.609816
Name: data1, dtype: float64

In [10]:
gs = df.data1.groupby(df.key1)
gs

In [11]:
print("="*50)
for n, g in gs:
    print("[key]:", n)
    print("[group]:", type(g))
    print("-"*50)
    print(g)
    print("-"*50)
    print("[mean]:", g.mean())
    print("="*50)

[key]: a
[group]: <class 'pandas.core.series.Series'>
--------------------------------------------------
0    1.764052
1    0.400157
4    1.867558
Name: data1, dtype: float64
--------------------------------------------------
[mean]: 1.343922514828285
[key]: b
[group]: <class 'pandas.core.series.Series'>
--------------------------------------------------
2    0.978738
3    2.240893
Name: data1, dtype: float64
--------------------------------------------------
[mean]: 1.6098155916535986


In [12]:
gs.mean()

key1
a    1.343923
b    1.609816
Name: data1, dtype: float64

- 문제: 복합 key (key1, key2) 값에 따른 data1의 평균은?

In [13]:
means = df.data1.groupby([df.key1, df.key2]).mean()
means

key1  key2
a     one     1.815805
      two     0.400157
b     one     0.978738
      two     2.240893
Name: data1, dtype: float64

### groupby 명령의 인수

- groupby 명령에서 Key 인수로 입력할 수 있는 값은 다음과 같다.
    - 열 또는 열의 리스트
    - 행 인덱스
    - 사전/함수: Column의 값을 사전에 매핑(mapping)하거나 함수 처리하여 나온 결괏값을 키로 인식

In [14]:
np.random.seed(0)
people = pd.DataFrame(np.random.randn(5, 5), 
                      columns=['a', 'b', 'c', 'd', 'e'], 
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.ix[2:3, ['b', 'c']] = np.nan
people

,a,b,c,d,e
Joe,1.764052,0.400157,0.978738,2.240893,1.867558
Steve,-0.977278,0.950088,-0.151357,-0.103219,0.410599
Wes,0.144044,NaN,NaN,0.121675,0.443863
Jim,0.333674,1.494079,-0.205158,0.313068,-0.854096
Travis,-2.552990,0.653619,0.864436,-0.742165,2.269755


In [15]:
print("="*80)
for n, g in people.groupby(people.index):
    print("[key]:", n)
    print("[group]:", type(g))
    print("-"*80)
    print(g)
    print("="*80)

[key]: Jim
[group]: <class 'pandas.core.frame.DataFrame'>
--------------------------------------------------------------------------------
            a         b         c         d         e
Jim  0.333674  1.494079 -0.205158  0.313068 -0.854096
[key]: Joe
[group]: <class 'pandas.core.frame.DataFrame'>
--------------------------------------------------------------------------------
            a         b         c         d         e
Joe  1.764052  0.400157  0.978738  2.240893  1.867558
[key]: Steve
[group]: <class 'pandas.core.frame.DataFrame'>
--------------------------------------------------------------------------------
              a         b         c         d         e
Steve -0.977278  0.950088 -0.151357 -0.103219  0.410599
[key]: Travis
[group]: <class 'pandas.core.frame.DataFrame'>
--------------------------------------------------------------------------------
              a         b         c         d         e
Travis -2.55299  0.653619  0.864436 -0.742165  2.269755

In [16]:
mapping = {'Joe': 'J', 'Jim': 'J', 'Steve': 'S', 'Wes': 'S', 'Travis': 'S'}
print("="*80)
for n, g in people.groupby(mapping):
    print("[key]:", n)
    print("[group]:", type(g))
    print("-"*80)
    print(g)
    print("="*80)

[key]: J
[group]: <class 'pandas.core.frame.DataFrame'>
--------------------------------------------------------------------------------
            a         b         c         d         e
Joe  1.764052  0.400157  0.978738  2.240893  1.867558
Jim  0.333674  1.494079 -0.205158  0.313068 -0.854096
[key]: S
[group]: <class 'pandas.core.frame.DataFrame'>
--------------------------------------------------------------------------------
               a         b         c         d         e
Steve  -0.977278  0.950088 -0.151357 -0.103219  0.410599
Wes     0.144044       NaN       NaN  0.121675  0.443863
Travis -2.552990  0.653619  0.864436 -0.742165  2.269755


In [17]:
cap1 = lambda x: x[0].upper()
print("="*80)
for n, g in people.groupby(cap1):
    print("[key]:", n)
    print("[group]:", type(g))
    print("-"*80)
    print(g)
    print("="*80)

[key]: J
[group]: <class 'pandas.core.frame.DataFrame'>
--------------------------------------------------------------------------------
            a         b         c         d         e
Joe  1.764052  0.400157  0.978738  2.240893  1.867558
Jim  0.333674  1.494079 -0.205158  0.313068 -0.854096
[key]: S
[group]: <class 'pandas.core.frame.DataFrame'>
--------------------------------------------------------------------------------
              a         b         c         d         e
Steve -0.977278  0.950088 -0.151357 -0.103219  0.410599
[key]: T
[group]: <class 'pandas.core.frame.DataFrame'>
--------------------------------------------------------------------------------
              a         b         c         d         e
Travis -2.55299  0.653619  0.864436 -0.742165  2.269755
[key]: W
[group]: <class 'pandas.core.frame.DataFrame'>
--------------------------------------------------------------------------------
            a   b   c         d         e
Wes  0.144044 NaN NaN  0

In [18]:
print("="*80)
for n, g in people.groupby(people.columns, axis=1):
    print("[key]:", n)
    print("[group]:", type(g))
    print("-"*80)
    print(g)
    print("="*80)

[key]: a
[group]: <class 'pandas.core.frame.DataFrame'>
--------------------------------------------------------------------------------
               a
Joe     1.764052
Steve  -0.977278
Wes     0.144044
Jim     0.333674
Travis -2.552990
[key]: b
[group]: <class 'pandas.core.frame.DataFrame'>
--------------------------------------------------------------------------------
               b
Joe     0.400157
Steve   0.950088
Wes          NaN
Jim     1.494079
Travis  0.653619
[key]: c
[group]: <class 'pandas.core.frame.DataFrame'>
--------------------------------------------------------------------------------
               c
Joe     0.978738
Steve  -0.151357
Wes          NaN
Jim    -0.205158
Travis  0.864436
[key]: d
[group]: <class 'pandas.core.frame.DataFrame'>
--------------------------------------------------------------------------------
               d
Joe     2.240893
Steve  -0.103219
Wes     0.121675
Jim     0.313068
Travis -0.742165
[key]: e
[group]: <class 'pandas.core.frame.

In [19]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f' : 'orange'}
for n, g in people.groupby(mapping, axis=1):
    print("[key]:", n)
    print("[group]:", type(g))
    print("-"*80)
    print(g)
    print("="*80)

[key]: blue
[group]: <class 'pandas.core.frame.DataFrame'>
--------------------------------------------------------------------------------
               c         d
Joe     0.978738  2.240893
Steve  -0.151357 -0.103219
Wes          NaN  0.121675
Jim    -0.205158  0.313068
Travis  0.864436 -0.742165
[key]: red
[group]: <class 'pandas.core.frame.DataFrame'>
--------------------------------------------------------------------------------
               a         b         e
Joe     1.764052  0.400157  1.867558
Steve  -0.977278  0.950088  0.410599
Wes     0.144044       NaN  0.443863
Jim     0.333674  1.494079 -0.854096
Travis -2.552990  0.653619  2.269755


## 특별한 group 별 연산

- 통계
    - describe()
- 그룹을 대표하는 하나의 값을 계산
    - agg(), aggregate()
- 대표값으로 필드를 교체
    - transform()
- 그룹 전체를 변형하는 계산
    - apply()